In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import tensorflow as tf
import csv

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [269]:
# housing_csv = pd.read_csv('../data/housing.csv')
new_housing_csv = pd.read_csv('../data/new_housing.csv')

#Length
print(len(new_housing_csv))

#Mask Sale Price
mask = (new_housing_csv["SALE PRICE"]) > 100000.0
new_housing_csv = new_housing_csv[mask]


#Mask for Sold after 2010
new_housing_csv['SALE DATE'] = pd.to_datetime(new_housing_csv['SALE DATE'])
mask = (new_housing_csv["SALE DATE"]).dt.year > 2010
new_housing_csv = new_housing_csv[mask]

#Length
print(len(new_housing_csv))

#Only important rows
new_housing_csv = new_housing_csv[["LAND VAL", "PARCEL VAL", "IMPROVE VAL", "SALE PRICE", "SQFT", "ROOMS", "BEDROOM", "BATH", "LIVING AREA", "GROSS AREA", "YEAR"]]


#Remove Nulls
mask = new_housing_csv.notnull().all(axis=1)
new_housing_csv = new_housing_csv.loc[mask, :]
print(len(new_housing_csv))

#Remove 0s
mask = (new_housing_csv != 0).all(axis=1)
new_housing_csv = new_housing_csv.loc[mask, :]


#Length
print(len(new_housing_csv))


housing_data = new_housing_csv[["LAND VAL", "PARCEL VAL", "SQFT", "ROOMS", "BEDROOM", "BATH", "LIVING AREA", "GROSS AREA", "YEAR"]]
housing_label = new_housing_csv[["SALE PRICE"]]



# print(new_housing_csv)

# housing_data = housing_csv[["housing_median_age", "total_rooms", "total_bedrooms", "median_income", "population", "households"]]
# housing_label = housing_csv[["median_house_value"]]


/var/folders/p6/bmvt9pvd0vd5t3yfr1yhfh580000gn/T/ipykernel_85846/2212246803.py:2: DtypeWarning: Columns (13,20,53,54,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  new_housing_csv = pd.read_csv('../data/new_housing.csv')


107899
17728
17087
6215


In [270]:
X = housing_data
Y = housing_label

In [271]:
mu = np.mean(X, axis=0)
sigma = np.std(X, axis=0)
X = (X - mu) / sigma

mu = np.mean(Y, axis=0)
sigma = np.std(Y, axis=0)
Y = (Y - mu) / sigma

X = pd.DataFrame.to_numpy(X)
Y = pd.DataFrame.to_numpy(Y)


In [272]:
class LinearRegression(tf.keras.Model):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation='relu', input_shape=[5])
        self.dense2 = tf.keras.layers.Dense(units=32, activation='relu')
        self.dense3 = tf.keras.layers.Dense(units=1, kernel_regularizer=tf.keras.regularizers.l2(0.01))
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [273]:
model = LinearRegression()

# Define the loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [274]:
x = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(Y, dtype=tf.float32)
print(len(x))
x_train = x[:5000]
y_train = y[:5000]
x_test = x[5001:6215]
y_test = y[5001:6215]
# Remove all instances where data is empty

6215


In [275]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    with tf.GradientTape() as tape:
        y_pred = model(x_train)
        loss = loss_fn(y_train, y_pred)
    
    # Backward and optimize
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Print progress
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.4f}")

Epoch [100/10000], Loss: 0.4558
Epoch [200/10000], Loss: 0.4349
Epoch [300/10000], Loss: 0.4251
Epoch [400/10000], Loss: 0.4183
Epoch [500/10000], Loss: 0.4129
Epoch [600/10000], Loss: 0.4085
Epoch [700/10000], Loss: 0.4046
Epoch [800/10000], Loss: 0.4011
Epoch [900/10000], Loss: 0.3978
Epoch [1000/10000], Loss: 0.3947
Epoch [1100/10000], Loss: 0.3917
Epoch [1200/10000], Loss: 0.3890
Epoch [1300/10000], Loss: 0.3860
Epoch [1400/10000], Loss: 0.3833
Epoch [1500/10000], Loss: 0.3807
Epoch [1600/10000], Loss: 0.3782
Epoch [1700/10000], Loss: 0.3758
Epoch [1800/10000], Loss: 0.3734
Epoch [1900/10000], Loss: 0.3710
Epoch [2000/10000], Loss: 0.3685
Epoch [2100/10000], Loss: 0.3662
Epoch [2200/10000], Loss: 0.3639
Epoch [2300/10000], Loss: 0.3617
Epoch [2400/10000], Loss: 0.3594
Epoch [2500/10000], Loss: 0.3571
Epoch [2600/10000], Loss: 0.3549
Epoch [2700/10000], Loss: 0.3527
Epoch [2800/10000], Loss: 0.3504
Epoch [2900/10000], Loss: 0.3482
Epoch [3000/10000], Loss: 0.3461
Epoch [3100/10000],

In [276]:
# print(model.layers[0].weights)

In [277]:
def accuracy(y_pred, y_true, sigma, mu, threshold=0.2):
    """
    Computes the accuracy of the predictions given the true labels, based on a threshold value.
    
    Parameters:
    y_pred (np.ndarray): An array of predicted labels.
    y_true (np.ndarray): An array of true labels.
    threshold (float): The threshold value to use for measuring accuracy.
    
    Returns:
    float: The accuracy of the predictions as a percentage.
    """
    # Compute the absolute difference between predicted and true values
    
    
    y_pred = (y_pred * sigma) + mu
    y_true = (y_true * sigma) + mu

    diff = np.abs(y_pred - y_true)
    print(y_pred)
    print(y_true)
    
    # Compute accuracy
    acc = np.mean(diff <= threshold * y_pred)
    
    # Convert to percentage
    acc_pct = acc * 100
    
    return acc_pct

In [278]:
pred = model(x_test)
loss = loss_fn(y_test, pred)
accuracy(pred, y_test, sigma, mu)

tf.Tensor(
[[ 542774.9 ]
 [ 762118.25]
 [ 695176.  ]
 ...
 [1013042.9 ]
 [1320091.  ]
 [1552182.9 ]], shape=(1214, 1), dtype=float32)
tf.Tensor(
[[ 445000.]
 [ 669000.]
 [ 657000.]
 ...
 [ 810000.]
 [ 950000.]
 [1690000.]], shape=(1214, 1), dtype=float32)


57.49588138385503

In [279]:
weights, bias = model.layers[0].get_weights()
print(weights)


[[ 0.11458933  0.12156705  0.11700203 ...  0.00323182  0.03688697
   0.02554604]
 [-0.1705997  -0.1211534   0.13011625 ... -0.26055804  0.1319434
   0.01007314]
 [ 0.22833017 -0.24231347  0.00414997 ... -0.17752418 -0.0524627
  -0.19856492]
 ...
 [-0.18502845  0.04064478  0.1503381  ... -0.12790845 -0.15697888
   0.11542995]
 [-0.17118928 -0.13320403 -0.06279024 ... -0.20220156 -0.08900296
   0.28249234]
 [-0.07024302  0.17712931  0.09056931 ...  0.06407394 -0.1726234
   0.12277757]]
